<a href="https://colab.research.google.com/github/janani1424/Hybrid-CNN-Model-for-Brain-Tumor-Detection/blob/main/local_run_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/janani1424/Hybrid-CNN-Model-for-Brain-Tumor-Detection.git

Cloning into 'Hybrid-CNN-Model-for-Brain-Tumor-Detection'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
!pip install gradio

In [ ]:
import random

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# Set the path to the dataset
dataset_path = "/content/drive/MyDrive/DATASET"

# Define the training and testing directories
train_dir = os.path.join(dataset_path, "/content/drive/MyDrive/DATASET/Training")
test_dir = os.path.join(dataset_path, "/content/drive/MyDrive/DATASET/Testing")

# Define the categories
categories = ["glioma", "meningioma", "notumor", "pituitary"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the categories
categories = ["glioma", "meningioma", "notumor", "pituitary"]

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

test_datagen = ImageDataGenerator(rescale=1./255)

target_size = (224, 224)
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

Found 5712 images belonging to 4 classes.
Found 1268 images belonging to 4 classes.


In [ ]:
# Load Pretrained Models
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Freeze Initial Layers
for layer in resnet.layers[:100]:
    layer.trainable = False
for layer in vgg16.layers[:15]:
    layer.trainable = False

In [ ]:
# Feature Extraction Layers
resnet_output = GlobalAveragePooling2D()(resnet.output)
vgg16_output = GlobalAveragePooling2D()(vgg16.output)

In [ ]:
# Concatenate Features
merged_features = Concatenate()([resnet_output, vgg16_output])

In [ ]:
# Fully Connected Layers
x = Dense(512, activation='relu')(merged_features)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
out = Dense(len(categories), activation='softmax')  # 4-class classification

In [ ]:
# Define Hybrid CNN Model
model = Model(inputs=[resnet.input, vgg16.input], outputs=out(x)) # Pass the output tensor from the 'out' layer

In [ ]:
# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# # Class labels
# classes = ['Meningioma', 'Glioma', 'Pituitary', 'No Tumor']

# stages = {'Meningioma': 'Stage 1-3', 'Glioma': 'Stage 1-4', 'Pituitary': 'Non-staged', 'No Tumor': 'N/A'}

# recovery_methods = {
#     'Meningioma': "...",
#     'Glioma': "...",
#     'Pituitary': "...",
#     'No Tumor': "..."
# }

# def preprocess_image(image):
#     image = cv2.resize(image, target_size)
#     image = img_to_array(image) / 255.0
#     return np.expand_dims(image, axis=0)

# def predict_tumor(image):
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     processed_image = preprocess_image(image)
#     prediction = model.predict([processed_image, processed_image])
#     class_idx = np.argmax(prediction)
#     tumor_type = classes[class_idx]
#     stage = stages.get(tumor_type, 'Unknown')
#     recovery = recovery_methods.get(tumor_type, 'No specific recovery steps available.')

#     result = f"### AI-Powered Brain Tumor Diagnosis 🤖\n\n**Tumor Type Identified:** {tumor_type}\n\n**Stage:** {stage}\n\n### Suggested Recovery Plan\n{recovery}"
#     return result

# demo = gr.Interface(
#     fn=predict_tumor,
#     inputs=gr.Image(type="numpy", label="Upload Your MRI Scan 📸"),
#     outputs=gr.Markdown(),
#     title="🧠 HYBRID CNN MODEL FOR BRAIN TUMOR DETECTION AND CLASSIFICATION",
#     description="Leverage AI-driven MRI analysis for tumor classification, staging, and personalized recovery insights.",
#     theme="huggingface",
#     live=True,
# )

# if __name__ == "__main__":
#     demo.launch()

In [ ]:
import gradio as gr

# Tumor details dictionary with added symptoms and treatment
tumor_data = {
    "Glioma": {
        "about": [
           "Gliomas originate from glial cells in the brain or spine.",
            "They can be benign or malignant with varying aggressiveness.",
            "Symptoms include headaches, seizures, and cognitive dysfunction.",
            "Treatment often includes surgery, radiation, and chemotherapy.",
            "Regular MRI scans are recommended for monitoring."
        ],
        "symptoms": [
            "Persistent headaches.",
            "Seizures or convulsions.",
            "Nausea and vomiting.",
            "Cognitive or personality changes.",
            "Weakness or numbness in limbs."
        ],
        "treatment": [
            "Surgical resection is typically the first line of treatment.",
            "Radiation therapy is used post-surgery or for inoperable tumors.",
            "Chemotherapy may be required for high-grade gliomas.",
            "Targeted therapies or clinical trials may be considered."
        ],
        "recovery": [
            "Surgical removal is typically the first step.",
            "Radiation therapy targets any remaining tumor tissue.",
            "Chemotherapy may be needed for aggressive gliomas.",
            "Frequent neurological evaluations help track progress.",
            "Healthy lifestyle choices aid overall recovery."
        ]
    },
    "Meningioma": {
        "about": [
            "Meningiomas arise from the meninges, the membranes covering the brain and spinal cord.",
            "Most meningiomas are non-cancerous but can grow large and cause symptoms.",
            "Common symptoms include vision problems, headaches, and memory issues.",
            "Diagnosis often involves MRI or CT imaging.",
            "Not all meningiomas require immediate treatment."
        ],
        "symptoms": [
            "Headaches, often worse in the morning.",
            "Blurred or double vision.",
            "Hearing loss or ringing in ears.",
            "Memory problems or personality changes.",
            "Seizures or focal neurological deficits."
        ],
        "treatment": [
            "Observation with periodic imaging if asymptomatic.",
            "Surgical removal for symptomatic or growing tumors.",
            "Radiation therapy if surgery isn’t complete or feasible.",
            "Stereotactic radiosurgery for small, hard-to-reach tumors."
        ],
        "recovery": [
            "Observation may be enough for small, asymptomatic tumors.",
            "Surgical resection is used when tumors are causing symptoms.",
            "Radiotherapy can be used for incompletely removed tumors.",
            "Follow-up imaging is required to detect regrowth.",
            "Supportive therapy improves quality of life."
        ]
    },
    "Pituitary": {
        "about": [
           "Pituitary tumors develop in the pituitary gland at the brain’s base.",
            "They can affect hormone production and cause systemic effects.",
            "Symptoms include hormonal imbalances, vision issues, and fatigue.",
            "Often detected via blood tests and MRI scans.",
            "May be functional (hormone-producing) or non-functional."
        ],
        "symptoms": [
            "Vision changes like tunnel vision or blurry vision.",
            "Unexplained weight gain or loss.",
            "Menstrual irregularities or sexual dysfunction.",
            "Fatigue, depression, or mood swings.",
            "Headaches behind the eyes or forehead."
        ],
        "treatment": [
            "Medication to control hormone levels or shrink the tumor.",
            "Transsphenoidal surgery (through the nose) for removal.",
            "Radiation therapy for residual or recurrent tumors.",
            "Hormone replacement therapy post-treatment if needed."
        ],
        "recovery": [
            "Medications can help regulate hormone levels.",
            "Surgery is recommended for larger or symptomatic tumors.",
            "Radiation therapy is used if tumors return.",
            "Hormone replacement therapy might be necessary long-term.",
            "Ongoing endocrine evaluations are crucial."
        ]
    },
    "No Tumor": {
        "about": [
            "The MRI scan appears to be free of detectable brain tumors.",
            "No abnormal growth or mass was identified in the image.",
            "This suggests a healthy brain structure.",
            "Symptoms may be due to non-tumor causes like migraine or stress.",
            "Consultation may still be advised if symptoms persist."
        ],
        "symptoms": [
            "Possible causes for symptoms: migraines, stress, infections.",
            "Tension headaches or sinus issues.",
            "Neurological symptoms may still warrant further testing."
        ],
        "treatment": [
            "No tumor-specific treatment is required.",
            "Treat symptoms as per underlying non-tumor condition.",
            "Lifestyle changes may alleviate symptoms."
        ],
        "recovery": [
            "No treatment is necessary for brain tumors.",
            "Maintain a healthy lifestyle and balanced nutrition.",
            "Regular exercise supports mental and physical health.",
            "Manage stress with meditation or therapy if needed.",
            "Stay hydrated and follow up with a physician if needed."
        ]
    }
}

# Simulated prediction function
def predict_tumor(img):
    tumor_type = random.choice(list(tumor_data.keys()))  # Simulate prediction
    tumor_info = "\n".join([f"{i+1}. {line}" for i, line in enumerate(tumor_data[tumor_type]["about"])])
    recovery_plan = "\n".join([f"{i+1}. {line}" for i, line in enumerate(tumor_data[tumor_type]["recovery"])])
    return tumor_type, tumor_info, recovery_plan

# Enhanced chatbot
def chat_with_bot(message, chat_history):
    message_lower = message.lower()
    response = "I'm here to help with information about brain tumors. "

    # Determine the tumor type being asked about
    for tumor in tumor_data:
        if tumor.lower() in message_lower:
            if "symptom" in message_lower:
                response = f"Symptoms of {tumor} include:\n" + "\n".join(
                    [f"- {s}" for s in tumor_data[tumor]["symptoms"]])
            elif "treatment" in message_lower or "treated" in message_lower:
                response = f"Treatment options for {tumor} include:\n" + "\n".join(
                    [f"- {t}" for t in tumor_data[tumor]["treatment"]])
            elif "recovery" in message_lower or "recover" in message_lower:
                response = f"Recovery process for {tumor}:\n" + "\n".join(
                    [f"- {r}" for r in tumor_data[tumor]["recovery"]])
            elif "about" in message_lower or "what is" in message_lower or "explain" in message_lower:
                response = f"Here's what you should know about {tumor}:\n" + "\n".join(
                    [f"- {a}" for a in tumor_data[tumor]["about"]])
            else:
                response = (
                    f"{tumor} Information:\n"
                    + "\n".join([f"- {a}" for a in tumor_data[tumor]["about"]])
                    + "\n\nSymptoms:\n"
                    + "\n".join([f"- {s}" for s in tumor_data[tumor]["symptoms"]])
                    + "\n\nTreatment:\n"
                    + "\n".join([f"- {t}" for t in tumor_data[tumor]["treatment"]])
                    + "\n\nRecovery:\n"
                    + "\n".join([f"- {r}" for r in tumor_data[tumor]["recovery"]])
                )
            break
    else:
        response = (
            "You can ask me about any of the following:\n"
            "- Symptoms of Glioma, Meningioma, Pituitary tumor, or No Tumor.\n"
            "- Treatment or recovery options for these tumors.\n"
            "- Explanation or general information about them.\n"
            "For example: 'What are the symptoms of Glioma?' or 'Explain Pituitary tumor.'"
        )

    chat_history.append((message, response))
    return "", chat_history

# Gradio UI
with gr.Blocks(css="body {background-color: #111;} .gr-image {width: 100% !important;}") as app:
    gr.Markdown("<h1 style='text-align: center; color: #ff44ff;'>HYBRID CNN MODEL FOR BRAIN TUMOR DETECTION AND CLASSIFICATION</h1>")

    with gr.Row():
        with gr.Column(scale=1.5):
            image_input = gr.Image(type="pil", label="Upload Your MRI Scan", height=550)
            with gr.Row():
                detect_btn = gr.Button("Detect Tumor", variant="primary")
                clear_btn = gr.Button("Clear", variant="secondary")

        with gr.Column(scale=2.5):
            gr.Markdown("<h2 style='color: #ff4444;'>Tumor Detected:</h2>")
            tumor_result = gr.Textbox(label="", interactive=False)

            gr.Markdown("<h2 style='color: #ffaa00;'>About Tumor:</h2>")
            tumor_details = gr.Textbox(label="", interactive=False, lines=7)

            gr.Markdown("<h2 style='color: #00aaff;'>Recovery Plan:</h2>")
            recovery_details = gr.Textbox(label="", interactive=False, lines=7)

    gr.Markdown("<hr><h2 style='color: #ffffff;'>💬 Ask Me Anything</h2>")
    chatbot = gr.Chatbot(label="Tumor Assistant Chatbot", height=300)
    chat_input = gr.Textbox(placeholder="Ask a question about brain tumors...", label="Your Question")

    chat_input.submit(chat_with_bot, inputs=[chat_input, chatbot], outputs=[chat_input, chatbot])
    detect_btn.click(predict_tumor, inputs=image_input, outputs=[tumor_result, tumor_details, recovery_details])
    clear_btn.click(lambda: (None, "", "", ""), outputs=[image_input, tumor_result, tumor_details, recovery_details])

app.launch()


/usr/local/lib/python3.11/dist-packages/gradio/layouts/column.py:55: UserWarning: 'scale' value should be an integer. Using 1.5 will cause issues.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/layouts/column.py:55: UserWarning: 'scale' value should be an integer. Using 2.5 will cause issues.
  warnings.warn(
<ipython-input-29-82360504dd97>:193: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Tumor Assistant Chatbot", height=300)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/04/23 05:55:49 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


<IPython.core.display.Javascript object>